In [7]:
import time
import stim

from surface_sim import Layout
from surface_sim.setup import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel
from surface_sim import Detectors
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
from surface_sim.layouts import unrot_surface_codes
import surface_sim.experiments.unrot_surface_code_css as exp
from qec_util.distance import get_circuit_distance


# INPUTS
EXPERIMENTS = ["I", "S", "H", "CNOT-alternating", "CNOT-no-alternating"]
DISTANCES = [3, 5]
NOISE_MODEL = CircuitNoiseModel
BASES = ["Z", "X"]
FRAMES = ["pre-gate", "post-gate"]

In [5]:
# GENERATE CIRCUIT
def get_num_qubits(experiment):
    return 1 if experiment in ["I", "S", "H", "SQRT_X"] else 2

def get_circuit(experiment, distance, basis):
    num_qubits = get_num_qubits(experiment)

    # reset
    circuit_str = ""
    if basis == "X":
        circuit_str = "RX " + " ".join([f"{i}" for i in range(num_qubits)]) + "\nTICK\n"
    else:
        circuit_str = "R " + " ".join([f"{i}" for i in range(num_qubits)]) + "\nTICK\n"

    # clifford gates + QEC cycles
    if num_qubits == 1:
        circuit_str += f"{experiment} 0\nTICK\n"*(distance+1)
    elif experiment == "CNOT-no-alternating":
        circuit_str += "CNOT 0 1\nTICK\n"*(distance+1)
    elif experiment == "CNOT-alternating":
        circuit_str += "CNOT 0 1\nTICK\nCNOT 1 0\nTICK\n"*((distance+1)//2)
    elif experiment == "CZ":
        circuit_str += "CZ 0 1\nTICK\n"*(distance+1)
    else:
        raise ValueError(f"{experiment} is not known.")

    # measurment
    if basis == "X":
        circuit_str += "MX " + " ".join([f"{i}" for i in range(num_qubits)])
    else:
        circuit_str += "M " + " ".join([f"{i}" for i in range(num_qubits)])

    circuit = stim.Circuit(circuit_str)

    return circuit

In [10]:
for experiment_name in EXPERIMENTS:
    for basis in BASES:
        for frame in FRAMES:
            for distance in DISTANCES:
                layouts = unrot_surface_codes(get_num_qubits(experiment_name), distance=distance)
                qubit_inds = {}
                anc_coords = {}
                anc_qubits = []
                stab_coords = {}
                for l, layout in enumerate(layouts):
                    qubit_inds.update(layout.qubit_inds())
                    anc_qubits += layout.get_qubits(role="anc")
                    coords = layout.anc_coords()
                    anc_coords.update(coords)
                    stab_coords[f"Z{l}"] = [v for k,v in coords.items() if k[0] == "Z"]
                    stab_coords[f"X{l}"] = [v for k,v in coords.items() if k[0] == "X"]

                circuit = get_circuit(experiment_name, distance, basis)

                setup = CircuitNoiseSetup()
                setup.set_var_param("prob", 1e-3)
                model = NOISE_MODEL(setup=setup, qubit_inds=qubit_inds)
                detectors = Detectors(anc_qubits, frame=frame, anc_coords=anc_coords)

                schedule = schedule_from_circuit(circuit, layouts, gate_to_iterator)
                experiment = experiment_from_schedule(
                    schedule, model, detectors, anc_reset=True, anc_detectors=None
                )
                print("start=", time.strftime("%D %H:%M:%S"))
                d_circ = get_circuit_distance(experiment)
                print(f"{experiment_name} b={basis} f={frame} d={distance} d_circ={d_circ}")
                print("finish=", time.strftime("%D %H:%M:%S"))

start= 02/06/25 16:02:04
I b=Z f=pre-gate d=3 d_circ=3
finish= 02/06/25 16:02:04
start= 02/06/25 16:02:09
I b=Z f=pre-gate d=5 d_circ=5
finish= 02/06/25 16:06:59
start= 02/06/25 16:07:00
I b=Z f=post-gate d=3 d_circ=3
finish= 02/06/25 16:07:00
start= 02/06/25 16:07:05
I b=Z f=post-gate d=5 d_circ=5
finish= 02/06/25 16:11:52
start= 02/06/25 16:11:53
I b=X f=pre-gate d=3 d_circ=3
finish= 02/06/25 16:11:54
start= 02/06/25 16:11:58
I b=X f=pre-gate d=5 d_circ=5
finish= 02/06/25 16:16:05
start= 02/06/25 16:16:06
I b=X f=post-gate d=3 d_circ=3
finish= 02/06/25 16:16:06
start= 02/06/25 16:16:11
I b=X f=post-gate d=5 d_circ=5
finish= 02/06/25 16:20:16
start= 02/06/25 16:20:17
S b=Z f=pre-gate d=3 d_circ=3
finish= 02/06/25 16:20:18
start= 02/06/25 16:20:23
S b=Z f=pre-gate d=5 d_circ=5
finish= 02/06/25 16:24:50
start= 02/06/25 16:24:51
S b=Z f=post-gate d=3 d_circ=3
finish= 02/06/25 16:24:51
start= 02/06/25 16:24:56
S b=Z f=post-gate d=5 d_circ=5
finish= 02/06/25 16:28:27
start= 02/06/25 16:28:

In [11]:
# INPUTS
EXPERIMENTS = ["S", "H"]
DISTANCES = [7]
NOISE_MODEL = CircuitNoiseModel
BASES = ["Z", "X"]
FRAMES = ["pre-gate", "post-gate"]

for experiment_name in EXPERIMENTS:
    for basis in BASES:
        for frame in FRAMES:
            for distance in DISTANCES:
                layouts = unrot_surface_codes(get_num_qubits(experiment_name), distance=distance)
                qubit_inds = {}
                anc_coords = {}
                anc_qubits = []
                stab_coords = {}
                for l, layout in enumerate(layouts):
                    qubit_inds.update(layout.qubit_inds())
                    anc_qubits += layout.get_qubits(role="anc")
                    coords = layout.anc_coords()
                    anc_coords.update(coords)
                    stab_coords[f"Z{l}"] = [v for k,v in coords.items() if k[0] == "Z"]
                    stab_coords[f"X{l}"] = [v for k,v in coords.items() if k[0] == "X"]

                circuit = get_circuit(experiment_name, distance, basis)

                setup = CircuitNoiseSetup()
                setup.set_var_param("prob", 1e-3)
                model = NOISE_MODEL(setup=setup, qubit_inds=qubit_inds)
                detectors = Detectors(anc_qubits, frame=frame, anc_coords=anc_coords)

                schedule = schedule_from_circuit(circuit, layouts, gate_to_iterator)
                experiment = experiment_from_schedule(
                    schedule, model, detectors, anc_reset=True, anc_detectors=None
                )
                print("start=", time.strftime("%D %H:%M:%S"))
                d_circ = get_circuit_distance(experiment)
                print(f"{experiment_name} b={basis} f={frame} d={distance} d_circ={d_circ}")
                print("finish=", time.strftime("%D %H:%M:%S"))

start= 02/07/25 03:29:01


error: Caught keyboard interrupt

In [ ]:
# it was taking more than a 12h to compute, so I stopped it